In [ ]:
api_key = ""
project = ""
region = ""
authorizationToken = ""

dataset_id = "basic_subclustering"
clusteringType = "community-detection"
cutoff = 0.75
encode_type = "text"
vector_fields = ["product_title_clip_vector_"]
n_clusters = 10


In [ ]:

## Instantiate client ###
from relevanceai import Client 
client = Client(token=authorizationToken)

## Checking valid vector field ###
for v in vector_fields:
  if not '_vector_'in v:
    raise ValueError(f"'{v}' is not a valid vector field")


ds = client.Dataset(dataset_id)
try:
  if clusteringType == 'community-detection':
    cluster_method  = "community_detection"
    ds.cluster(
        cluster_method,
        cluster_config={"threshold": cutoff},
        vector_fields=vector_fields
    )
  elif clusteringType == 'kmeans':
    if ds.shape[0] < 3000:
      ds.cluster(
          clusteringType,
          cluster_config={"n_clusters": n_clusters},
          vector_fields=vector_fields
      )
    else:
      cluster_method = 'minibatchkmeans'
      ds.cluster(
          cluster_method,
          cluster_config={"n_clusters": n_clusters},
          vector_fields=vector_fields
      )
  else:
    cluster_method = "kmeans"
    ds.cluster(
        cluster_method,
        cluster_config={"n_clusters": n_clusters},
        vector_fields=vector_fields
    )
except Exception as e:
  raise ValueError(f'{e}')

print("Finished clustering your data, you may close this window.")

